In [1]:
from geoclidean_env_euclid import *
from plot_utils import save_plot
import os
import random
import json

os.makedirs("data", exist_ok=True)

In [55]:
def make_rule_intersecting_circle():
    inlier = [
        'c1 = circle(p1(), p2())',
        'l3 = line(p3(c1), p4(c1))'
    ]

    outlier = [
        'c1 = circle(p1(), p2())',
        'l3 = line(p3(), p4())'
    ]
    return "circle_contains_intersecting_line", inlier, outlier

def make_rule_triangle_in_circle():
    inlier = [
        "c1 = circle(p1(), p2())",
        "l1 = line(p3(c1), p4(c1))",
        "l2 = line(p4(c1), p5(c1))",
        "l3 = line(p5(c1), p3(c1))"
    ]
    outlier = [
        "c1 = circle(p1(), p2())",
        "l1 = line(p3(c1), p4())",
        "l2 = line(p4(), p5(c1))",
        "l3 = line(p5(c1), p3(c1))"
    ]
    return "triangle_inscribed_in_circle", inlier, outlier

def make_rule_two_lines():
    inlier = [
        'c1 = circle(p1(), p2())',
        'l3 = line(p3(c1), p4(c1))',
        'l4 = line(p5(c1), p6(c1))',
    ]

    outlier = [
        'c1 = circle(p1(), p2())',
        'l3 = line(p3(c1), p4(c1))',
        'l4 = line(p5(), p6())',
    ]
    return "circle_contains_two_intersecting_lines", inlier, outlier

def make_rule_quadrilateral():
    inlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p4())',
        'l4 = line(p4(l3), p1(l1))',
    ]

    outlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p4())',
        'l4 = line(p4(l3), p5())',
    ]
    return "quadrilateral", inlier, outlier

def make_rule_triangle():
    inlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p1(l1))',
    ]

    outlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p4())',
    ]
    return "triangle", inlier, outlier

def make_rule_nonintersecting_lines():
    inlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p3(), p4())',
    ]

    outlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p3(l1), p4())',
    ]
    return "two_nonintersecting_lines", inlier, outlier

def make_rule_nested_circles():
    inlier = [
        'c1 = circle(p1(), p2())',
        'c2 = circle(p1(c1), p3())',
    ]

    outlier = [
        'c1 = circle(p1(), p2())',
        'c2 = circle(p3(), p4())',
    ]
    return "two_nested_circles", inlier, outlier

def make_rule_intersecting_nested_circle_line():
    inlier = [
        'c1 = circle(p1(), p2())',
        'c2 = circle(p3(c1), p4())',
        'l3 = line(p5(c1, c2), p6(c1, c2))'
    ]

    outlier = [
        'c1 = circle(p1(), p2())',
        'c2 = circle(p3(c1), p4())',
        'l3 = line(p5(c1), p6(c2))'
    ]
    return "two_circles_intersecting_line", inlier, outlier

def make_rule_triangle_intersecting():
    inlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p1(l1))',
        'l4 = line(p4(l1), p5(l2))'
    ]

    outlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p2(l1), p3())',
        'l3 = line(p3(l2), p1(l1))',
        'l4 = line(p4(), p5())'
    ]
    return "triangle_intersecting_line", inlier, outlier

def make_rule_3_nonintersecting_lines():
    inlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p1(), p4())',
        'l3 = line(p1(), p6())',
    ]

    outlier = [
        'l1 = line(p1(), p2())',
        'l2 = line(p1(), p4())',
        'l3 = line(p5(), p4())',
    ]
    return "three_intersecting_lines", inlier, outlier

In [56]:
# rule_functions = [make_rule_intersecting_circle, make_rule_triangle_in_circle]
rule_functions = [make_rule_3_nonintersecting_lines]

In [57]:
metadata = {}

num_rules = 10
for rule_id in range(1, num_rules + 1):
    rule_fn = random.choice(rule_functions)
    rule_text, inlier_dsl, outlier_dsl = rule_fn()

    rule_name = f"rule_{rule_id:03d}"
    folder = f"data/{rule_name}"
    os.makedirs(folder, exist_ok=True)

    image_paths = []

    # Generate 3 inliers
    for i in range(3):
        img_path = f"{folder}/inlier_{i}.png"
        generate_concept(inlier_dsl, path=img_path, show_plots=False, mark_points=False)
        image_paths.append({"filename": f"inlier_{i}.png", "label": "inlier"})

    # Generate 1 outlier
    img_path = f"{folder}/outlier.png"
    generate_concept(outlier_dsl, path=img_path, show_plots=False, mark_points=False)
    image_paths.append({"filename": "outlier.png", "label": "outlier"})

    metadata[rule_name] = {
        "rule": rule_text,
        "images": image_paths
    }

with open("data/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)